### BERT and RoBERTa training and testing

In [ ]:
#imports
import numpy as np
import pandas as pd
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer, BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments,TrainerCallback,AutoModel,AutoModelForSequenceClassification,BertConfig as BertConfig
import torch
import wandb
import evaluate
import gc

accuracy_metric = evaluate.load('accuracy')
precision_metric = evaluate.load('precision')
recall_metric = evaluate.load('recall')

In [6]:
def import_paths_and_nlabels(dataset_name,model_name,paragraph_selection_strategy):
    if dataset_name not in ['asylex-outcome','asylex-norp','sentiment1','sentiment2']:
        raise ValueError('dataset name not found')
    if model_name not in ['bert','roberta']:
        raise ValueError('model name not found')
    if paragraph_selection_strategy not in ['first','last','rand','cas','']:
        raise ValueError('long_text technique not found')

    base_dataset_path = 'datasets/' + model_name + '/'
    base_model_path = 'Models/'  + model_name + '/'

    dataset_filename = {'asylex-norp':'norp_','asylex-outcome':'outcome_','sentiment1':'sentiment1','sentiment2':'sentiment2'}
    model_filename = {'bert' : 'BERT512-', 'roberta' : 'RoBERTa512-'}
    model_filename2 = {'asylex-norp':'norp_','asylex-outcome':'out_','sentiment1':'sentiment1','sentiment2':'sentiment2'}

    if dataset_name == 'asylex-norp':
        num_labels = 7
    else:
        num_labels = 2

    dataset_path = base_dataset_path+ dataset_name+ '/' + dataset_filename[dataset_name] + paragraph_selection_strategy + '_'
    model_path = base_model_path + dataset_name + '/' + model_filename[model_name] + model_filename2[dataset_name] +paragraph_selection_strategy
    return dataset_path, model_path, num_labels

In [ ]:
#dataset import
dataset_name = 'asylex-outcome'
model_name = 'bert'
paragraph_selection_strategy = 'first'

dataset_path, trained_model_path, n_labels  = import_paths_and_nlabels(dataset_name,model_name,paragraph_selection_strategy)
train_set = load_from_disk(dataset_path+'train_set')
test_set = load_from_disk(dataset_path+'test_set')
validation_set = load_from_disk(dataset_path+'validation_set')

## Training

In [ ]:
#run if the model already exists and you want to test its performances
tokenizer = AutoTokenizer.from_pretrained(trained_model_path)
model = AutoModelForSequenceClassification.from_pretrained(trained_model_path, num_labels=n_labels)

In [ ]:
#choose this or the following (BERT or RoBERTa)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=n_labels)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = AutoModelForSequenceClassification.from_pretrained('roberta-base', num_labels=n_labels)

In [9]:
my_lr = 5e-6
my_epochs = 1

In [ ]:
wandb.init(
    project="Testing pc version",
    name= dataset_name+'_'+model_name+'_'+paragraph_selection_strategy,
    config={
        "learining_rate":my_lr,
        "batch_size":16,
        "epochs":my_epochs,
        "note":""
    }
)

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    precision = precision_metric.compute(predictions=predictions, references=labels)
    recall = recall_metric.compute(predictions=predictions, references=labels)
    results = {**accuracy,**precision,**recall}
    return results

training_args = TrainingArguments(
    output_dir="./results",          
    eval_strategy="epoch",    
    learning_rate=my_lr,             
    per_device_train_batch_size=4, 
    per_device_eval_batch_size=4,  
    num_train_epochs=my_epochs,             
    weight_decay=0.01,              
    logging_dir="./logs",           
    #logging_steps=3,
    logging_strategy="epoch",
    logging_first_step = True,
    log_level = "info",
    report_to = "wandb",
    save_strategy="epoch",
    save_safetensors= False
)

trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_set,         
    eval_dataset=validation_set,          
    compute_metrics = compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
wandb.finish()

## Testing

In [ ]:
trainer.evaluate(eval_dataset=test_set)

In [ ]:
model.save_pretrained(trained_model_path)
tokenizer.save_pretrained(trained_model_path)